In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


문제 1-1 : 기본 Chain 만들기 - AI 요리사


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "당신은 집밥 요리의 대가 입니다. 사용자가 가진 재료들을 이용해서 만들 수 있는 요리 한 가지를 추천하고, 그 요리의 레시피를 간단히 설명해주세요. 재료: {food}"

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="openai/gpt-oss-120b",  # Spring AI와 동일한 모델
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"food":"토마토, 양파, 치즈"})
pprint(response)

문제 1-2 : 2단계 Multi Chain 만들기 - 여행지 정보 시스템


In [ ]:
import os
from pprint import pprint
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="openai/gpt-oss-120b",  # Spring AI와 동일한 모델
    temperature=0.7
)

#첫번째 프롬프트
prompt_step1 = ChatPromptTemplate.from_messages([
    ("system", "당신은 세계 최고의 여행 가이드입니다. 사용자가 입력한 여행지의 가장 대표적인 관광 명소 딱 한 곳의 이름만 간결하게 추천해주세요."),
    ("user", "여행지: {destination}")
])

chain_step1 = prompt_step1 | llm | StrOutputParser()

#두번째 프롬프트
prompt_step2 = ChatPromptTemplate.from_messages([
    ("system", "당신은 관광 명소에 대한 풍부한 지식을 가진 역사학자이자 여행 작가입니다."),
    ("user", "추천받은 관광 명소인 '{landmark}'에 대해 아래 형식에 맞춰 자세히 설명해주세요.\n\n- 역사:\n- 특징:\n- 방문 팁:")
])

chain_step2 = prompt_step2 | llm | StrOutputParser()


# 1. 사용자 입력 정의
destination = "로마"
print(f"## 입력: {destination}\n")
print("---")

# 2. 1단계 체인을 먼저 실행하고 결과를 변수에 저장
print("## 1단계 실행 결과")
landmark_result = chain_step1.invoke({"destination": destination})
print(f"추천 명소: {landmark_result}\n")
print("---")

# 3. 1단계 결과를 2단계 체인의 입력으로 사용하여 실행
print("## 2단계 실행 결과")
details_result = chain_step2.invoke({"landmark": landmark_result})
# pprint를 사용하면 긴 텍스트의 줄바꿈이 깔끔하게 처리됩니다.
pprint(details_result)

문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용 


In [ ]:
import os
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser


# LLM 설정 (Groq API 사용)
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="openai/gpt-oss-120b",  # Spring AI와 동일한 모델
    temperature=0.7
)

# Few-Shot 학습을 위한 예시 데이터
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "대한민국 축구 대표팀이 월드컵 최종 예선에서 극적인 승리를 거두며 본선 진출의 희망을 이어갔다. 손흥민 선수는 결승골을 기록하며 팀을 승리로 이끌었다.",
        "keywords": "축구대표팀, 월드컵, 손흥민"
    }
]

# 각 예시의 형식을 정의하는 프롬프트 템플릿
# 사용자의 질문(news)과 AI의 답변(keywords) 형식을 지정합니다.
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# Few-Shot 프롬프트 템플릿 생성
# 위에서 정의한 예시 데이터와 형식을 few-shot 템플릿으로 묶습니다.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 기사의 내용을 정확히 파악하여 핵심 키워드 3개를 추출하는 전문가입니다. 반드시 '키워드: ' 형식으로만 답변해야 합니다."),
    few_shot_prompt,
    ("human", "{input}")
])

# LCEL을 사용하여 전체 체인 구성
chain = final_prompt | llm | StrOutputParser()

# 7. 테스트할 새로운 뉴스 기사
test_news = "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."

# 8. 체인 실행 및 결과 출력
response = chain.invoke({"input": test_news})

print("테스트 뉴스 원문\n")
pprint(test_news)
print("\n" + "="*50 + "\n")
print(response)